In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_columns

### Creating the working set (FAO Data)

In [ ]:
crop_harvest = pd.read_csv("./raw_data/fao_data/fao_harvest.csv")
crop_prod = pd.read_csv("./raw_data/fao_data/fao_prod.csv")
crop_yield = pd.read_csv("./raw_data/fao_data/fao_yield.csv")
change_temp = pd.read_csv("./raw_data/fao_data/fao_temp.csv")

In [ ]:
crop_harvest = crop_harvest.loc[(crop_harvest["Flag"] == "A") | (crop_harvest["Flag"] == "E")]
crop_harvest = crop_harvest.reset_index(drop=True)
crop_harvest = crop_harvest.drop_duplicates()

crop_prod = crop_prod.loc[(crop_prod["Flag"] == "A") | (crop_prod["Flag"] == "E")]
crop_prod = crop_prod.reset_index(drop=True)
crop_prod = crop_prod.drop_duplicates()

crop_yield = crop_yield.loc[(crop_yield["Flag"] == "A") | (crop_yield["Flag"] == "E")]
crop_yield = crop_yield.reset_index(drop=True)
crop_yield = crop_yield.drop_duplicates()
crop_yield["Value"] = crop_yield["Value"].apply(lambda x: x / 10_000)

change_temp = change_temp.loc[(change_temp["Flag"] == "A") | (change_temp["Flag"] == "E")]
change_temp = change_temp.reset_index(drop=True)
change_temp = change_temp.drop_duplicates()
change_temp = change_temp[change_temp["Element"] == "Temperature change"]

In [ ]:
crop_harvest = crop_harvest.rename(columns={"Value":"Harvest (ha)"})
crop_prod = crop_prod.rename(columns={"Value":"Production (tonnes)"})
crop_yield = crop_yield.rename(columns={"Value":"Yield (tonnes/ha)"})
change_temp = change_temp.rename(columns={"Value":"Change (C)"})

In [ ]:
# Combines both dataframes and creates the final FAO dataframe
## Keep 5 columns, year item Harvest (ha) Production (tonnes) Yiel (tonnes/ha)
final_fao = crop_yield[["Year","Item", "Yield (tonnes/ha)"]] \
.merge(crop_harvest[["Year","Item", "Harvest (ha)"]], on=["Year", "Item"], how="inner")
final_fao = final_fao.merge(crop_prod[["Year","Item", "Production (tonnes)"]], on=["Year", "Item"], how="inner")
final_fao = final_fao.merge(change_temp[["Year", "Change (C)"]], on="Year", how="inner")
final_fao

In [ ]:
# Overlook of data
final_fao[(final_fao["Year"] == 2021)].sample(10)# & final_data["Item"] == "Wheat"]#.iloc[0]

### Creating the working Subset (DWD Data)

In [ ]:
precip = pd.read_csv("./raw_data/precipitation/observed_average_annual_precipitation.csv")
air_temp_mean = pd.read_csv("./raw_data/air_temperature/average_air_temperature.txt", sep=";", index_col=False)
frost_days = pd.read_csv("./raw_data/frost_days/regional_averages_tnas_year.txt", sep=";", skiprows=1)
hot_days = pd.read_csv("./raw_data/hot_days/regional_averages_txbs_year.txt", sep=";", skiprows=1)
ice_days = pd.read_csv("./raw_data/ice_days/regional_averages_txcs_year.txt", sep=";", skiprows=1)
summer_days = pd.read_csv("./raw_data/summer_days/regional_averages_txas_year.txt", sep=";", skiprows=1)#, index_col=False) 
sunshine_duration = pd.read_csv("./raw_data/sunshine_duration/regional_averages_sd_year.txt", sep=";", skiprows=1) # hours
tropical_nights = pd.read_csv("./raw_data/tropical_nights/regional_averages_tnes_year.txt", sep=";",skiprows=1)

In [ ]:
# Changing hours to days
sunshine_duration["Deutschland"] = sunshine_duration["Deutschland"].apply(lambda x: (x / 24) / 365)

In [ ]:
# creating sub dataframes for ease of use
de_pre = precip[["Category", "Annual Mean"]]
de_pre = de_pre.rename(columns={"Annual Mean":"Annual Mean Precipitation"})
de_temp = air_temp_mean[["Jahr", "Deutschland"]]
de_temp = de_temp.rename(columns={"Deutschland":"Air Temp Mean"})
de_frost = frost_days[["Jahr", "Deutschland"]]
de_frost = de_frost.rename(columns={"Deutschland":"Frost Days"})
de_hot = hot_days[["Jahr", "Deutschland"]]
de_hot = de_hot.rename(columns={"Deutschland":"Hot Days"})
de_ice = ice_days[["Jahr", "Deutschland"]]
de_ice = de_ice.rename(columns={"Deutschland":"Ice Days"})
de_sum = summer_days[["Jahr", "Deutschland"]]
de_sum = de_sum.rename(columns={"Deutschland":"Summer Days"})
de_sun = sunshine_duration[["Jahr", "Deutschland"]]
de_sun = de_sun.rename(columns={"Deutschland":"Sunshine Duration"})
de_trop = tropical_nights[["Jahr", "Deutschland"]]
de_trop = de_trop.rename(columns={"Deutschland":"Tropical Nights"})

In [ ]:
## Merging all of the sub dataframes
final_dwd = de_pre.merge(de_temp, how="inner", left_on="Category", right_on="Jahr").merge(de_frost, how="inner").\
                    merge(de_hot, how="inner").merge(de_ice, how="inner").merge(de_sum, how="inner").merge(de_sun, how="inner").merge(de_trop, how="inner")

In [ ]:
# master_dwd = master_dwd.set_index("Jahr", drop=True)
final_dwd = final_dwd.drop(columns=["Category"])
final_dwd = final_dwd.rename(columns={"Jahr":"Year"})

In [ ]:
final_dwd

### Random Data

In [ ]:
co2 = np.loadtxt(r"./raw_data/co2_mm_mlo.txt")
co2 = pd.DataFrame(data=co2, columns=["Year","Month","decimal date", "monthly average", "de-seasonalized", " number of days", "st.dev of days", "unc. of monthly mean"])
co2["Year"] = co2["Year"].astype(int)

In [ ]:
co2.dtypes

In [ ]:
co2 = co2[["Year", "monthly average"]]
co2 = co2.groupby("Year", as_index=False).mean()

In [ ]:
co2

### Merging all datasets (FAO DWD RANDOM)

In [ ]:
final_data = final_fao.merge(final_dwd, how="inner", left_on="Year", right_on="Year")
final_data = final_data.merge(co2, how="inner", left_on="Year", right_on="Year")

In [ ]:
final_data = final_data.sort_values(by=["Year", "Item"], ignore_index=True)
final_data

## Saving the final dataframe into a csv

In [ ]:
final_data.to_csv("dataset.csv")